<a href="https://colab.research.google.com/github/hagabbar/craft_prospect/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import required scripts to run
import numpy as np
from keras import Sequential
from keras.layers import LSTM, Dense, Embedding, SpatialDropout1D


Using TensorFlow backend.


In [9]:
"""
This is a script which will check for non-random subsequences 
from the quantum random number generator onboard the CubeSat. 
The code will first perform a benchmark suite of tests known 
as Diehard tests. We will then use the results from these 
tests in order to train and compare the results of a 
neural network approach (LSTMs).
"""

# Define important global variables here
tot_num_samples = 10000 # total number of training samples (50/50 split between rand and non-rand)
train_split = int(int(tot_num_samples/2) * 0.9) # use 90% for training, 10% for testing
bitstream_len = 32     # length of observation window of network
batch_size = 8         # batch size (how many samples NN sees during each iteration)
#true_rand_url = "https://raw.githubusercontent.com/hagabbar/craft_prospect/master/key_assurance/true_random_nums.txt"

# Generate a set of pseudo random numbers
pseudo_rand_data_train = np.random.randint(low=0,high=4294967295,size=(train_split,bitstream_len),dtype=np.uint32)
pseudo_rand_data_test = np.random.randint(low=0,high=4294967295,size=(int(tot_num_samples/2) - train_split,bitstream_len),dtype=np.uint32)
pseudo_rand_labels_train = np.ones(train_split)
pseudo_rand_labels_test = np.ones(int(tot_num_samples/2) - train_split)

# Inject noise signal into pseudo random number sequences (sine-wave in binary)
time = np.arange(0, bitstream_len, 1)
#nonrand_seq = [0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1]
#nonrand_seq = np.random.shuffle(nonrand_seq)
nonrand_seq = (np.sin(time) + 1)
nonrand_seq = nonrand_seq / np.max(nonrand_seq)
noise_data_train = np.float32(np.random.randint(low=0,high=4294967295,size=(train_split,bitstream_len),dtype=np.uint32))
noise_data_train *= nonrand_seq
noise_data_train = np.uint32(noise_data_train)
noise_data_test = np.random.randint(low=0,high=4294967295,size=(int(tot_num_samples/2) - train_split,bitstream_len),dtype=np.uint32)
noise_labels_train = np.zeros(train_split)
noise_labels_test = np.zeros(int(tot_num_samples/2) - train_split)

# Load in set of true random numbers
#true_rand_data_train = np.loadtxt(true_rand_url)[:train_split,:]
#true_rand_data_test = np.loadtxt(true_rand_url)[train_split:int(tot_num_samples/2),:]
#true_rand_labels_train = np.zeros(train_split)
#true_rand_labels_test = np.zeros(int(tot_num_samples/2) - train_split)

# Combine all data and labels into two arrays (x and y respectively)
X_train = np.concatenate((pseudo_rand_data_train,noise_data_train))
X_test = np.concatenate((pseudo_rand_data_test,noise_data_test))
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

Y_train = np.concatenate((pseudo_rand_labels_train,noise_labels_train))
Y_test = np.concatenate((pseudo_rand_labels_test,noise_labels_test))

# Randomly shuffle training sets
idx_shuffle = np.random.permutation(int(train_split*2))
X_train = X_train[idx_shuffle,:]
Y_train = Y_train[idx_shuffle]


In [3]:
print(pseudo_rand_data_train)
print(noise_data_train)

[[1044405885  626551865 1935494435 ... 3312029638 1032849502 2550944879]
 [ 281792314  882321293 3557584743 ... 3921409609 2866101252 1914469064]
 [3626599685 3846698827  142670720 ... 1893114851 3241699420 2762271397]
 ...
 [1980751426 4195199560 3199417216 ... 3282048840 2089833473  797262455]
 [4266174975 2211786312 3811260251 ... 4212686197  923874888 1324908491]
 [3389930013 1386279364 1659557759 ...  138951885 2843923368 2340269129]]
[[2135936768 2852101120 2285723136 ...  661030144   19473540  481469472]
 [ 139919200  979267904 3954193664 ...  221175216   16679766  192828336]
 [ 175472496 3272294656 2442852096 ...   79582104   18041874  110459352]
 ...
 [1058995904  631518400  139393152 ...  264165440   18923600 1182849280]
 [1015056448 3630254592 2247961088 ...  599228416   24134398  422871104]
 [1738650496 2416398848 1382792448 ...   28601078   17875894  124900296]]


In [10]:
# Set-up LSTM network
embed_dim = 128
lstm_out = 200
batch_size = 32

model = Sequential()
#model.add(LSTM(lstm_out, dropout_U = 0.2, return_sequences=True, dropout_W = 0.2))
#model.add(LSTM(lstm_out, dropout_U = 0.2, return_sequences=True, dropout_W = 0.2))
model.add(LSTM(lstm_out, activation='relu',input_shape=(bitstream_len,1)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'mse', optimizer='adam')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 200)               161600    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
Total params: 161,801
Trainable params: 161,801
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
# Train LSTM network
model.fit(X_train, Y_train, batch_size = batch_size, epochs = 5,  verbose = 5)

Epoch 1/5


InternalError: GPU sync failed

In [ ]:
score,acc = model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))